<a href="https://colab.research.google.com/github/Bishop1303/ML_FlightDelayClassifier/blob/master/ML_FlightDelayClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Getting the softwares:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar -xvf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark


# To use spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

# SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark


In [ ]:
#Carica il drive con i dati:
from google.colab import drive
drive.mount('/content/drive')

In [22]:
flights = spark.read.csv("/content/drive/My Drive/flights.csv", inferSchema=True, header=True, mode='FAILFAST')
flights.show()
flights.printSchema()

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65|   NA|
|  5|  2|  1|     UA|   704|SFO| 550|  7.98|     102|    2|
|  7|  2|  6|     AA|   380|ORD| 733| 10.83|     135|   54|
|  1| 16|  6|     UA|  1477|ORD|1440|   8.0|     232|   -7|
|  1| 22|  5|     UA|   620|SJC|1829|  7.98|     250|  -13|
| 11|  8|  1|     OO|  5590|SFO| 158|  7.77|      60|   88|
|  4| 26|  1|     AA|  1144|SFO|1464| 13.25|     210|  -10|
|  4| 25|  0|     AA|   321|ORD| 978| 13.75|     160|   31|
|  8| 30|  2|     UA|   646|ORD| 719| 13.28|     151|   16|
|  3| 16|  3|     UA|   107|ORD|1745|   

Tune the raw dataset:

1.   Removing an uninformative column, (**flight**).

2.   removing rows which do not have information about whether or not a flight was delayed, (condition on **delay** column).



In [71]:
# Remove the 'flight' column
flights_drop_column = flights.drop('flight')

# Remove records with missing 'delay' values or NA values
flights_none_missing = flights_drop_column.filter((flights_drop_column.delay.isNotNull()) & \
                                                  (flights_drop_column.delay != 'NA'))



flights_none_missing = flights_none_missing.withColumn('delay', flights_none_missing['delay'].cast('int'))

# Check on dataframe
print('The Schema is: ')
flights_none_missing.printSchema()
print('=====================================================')
print('Informative rows after dropping malformed: ',flights_none_missing.count())

The Schema is: 
root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: integer (nullable = true)

Informative rows after dropping malformed:  47022


Tweaking the data:


1.   Converting the units of distance, replacing the *mile* column with a *km* column
2.   Creating a Boolean column indicating whether or not a flight was delayed, (>15 mins = delayed)



In [72]:
# Import the required function
from pyspark.sql.functions import round

# Conversion: 'mile' to 'km' and drop 'mile' column
flights_km = flights_none_missing.withColumn('km', round(flights.mile * 1.60934, 0)).drop('mile')

# Creating 'label' column indicating whether flight delayed (1) or not (0)
flights_km = flights_km.withColumn('label', (flights_km.delay >= 15).cast('integer'))

# Check records
flights_km.show(5)
flights_km.printSchema()

+---+---+---+-------+---+------+--------+-----+------+-----+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|
+---+---+---+-------+---+------+--------+-----+------+-----+
|  0| 22|  2|     UA|ORD| 16.33|      82|   30| 509.0|    1|
|  2| 20|  4|     UA|SFO|  6.17|      82|   -8| 542.0|    0|
|  9| 13|  1|     AA|ORD| 10.33|     195|   -5|1989.0|    0|
|  5|  2|  1|     UA|SFO|  7.98|     102|    2| 885.0|    0|
|  7|  2|  6|     AA|ORD| 10.83|     135|   54|1180.0|    1|
+---+---+---+-------+---+------+--------+-----+------+-----+
only showing top 5 rows

root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- org: string (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- km: double (nullable = true)
 |-- label: integer (nullable = true)



Transforming categorical columns (**carrier**, **org**) in numerical values

In [73]:
from pyspark.ml.feature import StringIndexer

# Create an indexer
indexer = StringIndexer(inputCol='carrier', outputCol='carrier_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(flights_km)

# Indexer creates a new column with numeric index values
flights_indexed = indexer_model.transform(flights_km)

# Repeat the process for the other categorical feature
flights_indexed = StringIndexer(inputCol='org', outputCol='org_idx').fit(flights_indexed).transform(flights_indexed)

# Check result
flights_indexed.show()

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|carrier_idx|org_idx|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
|  0| 22|  2|     UA|ORD| 16.33|      82|   30| 509.0|    1|        0.0|    0.0|
|  2| 20|  4|     UA|SFO|  6.17|      82|   -8| 542.0|    0|        0.0|    1.0|
|  9| 13|  1|     AA|ORD| 10.33|     195|   -5|1989.0|    0|        1.0|    0.0|
|  5|  2|  1|     UA|SFO|  7.98|     102|    2| 885.0|    0|        0.0|    1.0|
|  7|  2|  6|     AA|ORD| 10.83|     135|   54|1180.0|    1|        1.0|    0.0|
|  1| 16|  6|     UA|ORD|   8.0|     232|   -7|2317.0|    0|        0.0|    0.0|
|  1| 22|  5|     UA|SJC|  7.98|     250|  -13|2943.0|    0|        0.0|    5.0|
| 11|  8|  1|     OO|SFO|  7.77|      60|   88| 254.0|    1|        2.0|    1.0|
|  4| 26|  1|     AA|SFO| 13.25|     210|  -10|2356.0|    0|        1.0|    1.0|
|  4| 25|  0|     AA|ORD| 13

Consolidate predictor columns into a single column

In [76]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=['mon',
    'dom', 'dow', 'carrier_idx', 'org_idx', 'km', 'depart', 'duration'
], outputCol='features')

# Consolidate predictor columns
flights_assembled = assembler.transform(flights_indexed)

# Check the resulting column
flights_assembled.select('features', 'delay').show(5, truncate=False)


+-----------------------------------------+-----+
|features                                 |delay|
+-----------------------------------------+-----+
|[0.0,22.0,2.0,0.0,0.0,509.0,16.33,82.0]  |30   |
|[2.0,20.0,4.0,0.0,1.0,542.0,6.17,82.0]   |-8   |
|[9.0,13.0,1.0,1.0,0.0,1989.0,10.33,195.0]|-5   |
|[5.0,2.0,1.0,0.0,1.0,885.0,7.98,102.0]   |2    |
|[7.0,2.0,6.0,1.0,0.0,1180.0,10.83,135.0] |54   |
+-----------------------------------------+-----+
only showing top 5 rows



# **Decision Tree**
root node: contains all the data

Child node: separate in 2 the main node based on classification criteria.

Recursive approch to every child node to keep splitting...

The decision tree must use **features** to predict **delay**


# **Split train/test**

Random split the main dataset in 2 sets: training and test. Usually the training set is 80% of the total data, 4 times more then the test set.

Splitting data is important: DO NOT TEST MODELS ON TRAINED DATA of course they will perfom well...

1.   Training data (used to train the model) about 80% of the total data
2.   Testing data (used to test the model) remaning 20% of the data.




In [78]:
# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights_assembled.randomSplit([0.8, 0.2], seed=7)

# Check that training set has around 80% of records
training_ratio = flights_train.count() / flights_test.count()
print('Training/Test data ratio is: ',training_ratio)

Training/Test data ratio is:  3.944479495268139


# Build a Decision Tree
Using the data: *flights_train* and *flights_test* to fit a **Decision Tree model**.



In [79]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import DecisionTreeClassifier

# Classifier object
tree = DecisionTreeClassifier()

# Fiting the training data
tree_model = tree.fit(flights_train)

# Create predictions for the testing data
prediction = tree_model.transform(flights_test)

# Check the predictions
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+---------------------------------------+
|label|prediction|probability                            |
+-----+----------+---------------------------------------+
|1    |0.0       |[0.5393553223388305,0.4606446776611694]|
|0    |1.0       |[0.3447961260439329,0.655203873956067] |
|1    |1.0       |[0.3411764705882353,0.6588235294117647]|
|1    |0.0       |[0.5393553223388305,0.4606446776611694]|
|1    |1.0       |[0.3447961260439329,0.655203873956067] |
+-----+----------+---------------------------------------+
only showing top 5 rows



# **Confusion Matrix**

A confusion matrix gives a useful breakdown of predictions versus known values. It has four cells which represent the counts of:

* **True Negatives** (**TN**) — model predicts negative outcome & known outcome is negative  

*   **True Positives** (**TP**) — model predicts positive outcome & known outcome is positive

* **False Negatives** (**FN**) — model predicts negative outcome but known outcome is positive

* **False Positives** (**FP**) — model predicts positive outcome but known outcome is negative.

$$\text{accuracy} :=\frac{(TN+TP)}{(TN+TP+FN+FP)}$$


In [84]:
# Confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label = 1').count()
FP = prediction.filter('prediction = 1 AND label = 0').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)

# Check accuracy
print('The model accuracy is: ', '{:.3f}'.format(accuracy))

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1365|
|    0|       0.0| 2586|
|    1|       1.0| 3512|
|    0|       1.0| 2047|
+-----+----------+-----+

The model accuracy is:  0.641


# Logistic Curve

In [86]:
# Import the logistic regression class
from pyspark.ml.classification import LogisticRegression

# Create a classifier object and train on training data
logistic = LogisticRegression().fit(flights_train)

# Create predictions for the testing data and show confusion matrix
prediction = logistic.transform(flights_test)

# Confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label = 1').count()
FP = prediction.filter('prediction = 1 AND label = 0').count()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1738|
|    0|       0.0| 2615|
|    1|       1.0| 3139|
|    0|       1.0| 2018|
+-----+----------+-----+



# **Evaluate the Logistic Regression model**

Accuracy is generally not a very reliable metric because it can be biased by the most common target class.



*   **precision**: is the proportion of positive predictions which are correct. For all flights which are predicted to be delayed, (i.e. what proportion is actually delayed?)
$$\text{precision} := \frac{TP}{(TP+FP)}$$


*   **recall**: is the proportion of positives outcomes which are correctly predicted. For all delayed flights, (i.e. what proportion is correctly predicted by the model?)
$$\text{recall} := \frac{TP}{(TP+FN)}$$


*The precision and recall are generally formulated in terms of the positive target class. But it's also possible to calculate weighted versions of these metrics which look at both target classes.*


In [91]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)

print('precision = {:.2f}\n recall = {:.2f}'.format(precision, recall))

# Find weighted precision
multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: 'weightedPrecision'})

# Find AUC
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction, {binary_evaluator.metricName: 'areaUnderROC'})

# AOC, should be near 1
print('The area under the curve is: ','{:.2f}'.format(auc))

precision = 0.61
 recall = 0.64
The area under the curve is:  0.65
